In [3]:
#4.常量，变量，占位符
#1.构建二元线性回归模型，其中模型中的参数w,b使用`tf.Variable()`构建，模型的样本输入x使用`tf.placeholder`代替
import tensorflow as tf

w=tf.Variable(10)
b=tf.Variable(10)

with tf.Session() as sess:
    sess.run(w.initializer)
    
with tf.Session() as sess:
    sess.run(b.initializer)
    
x = tf.placeholder(dtype=tf.int32, shape=[])#占位符必须包含数据类型!!!!!
y=tf.multiply(w,x)+b
#图构建好之后，运行图时，需要使用张量替代占位符，否则这个图无法运行
with tf.Session() as sess:
    print(sess.run(y, feed_dict={x: 5}))
    
#使用tf.constant()生成常量时，不能使用其他常量张量初始化
#各种常量类型包括普通常量、序列常量、随机数常量、特殊常量

In [4]:
#5.名字与作用域
#1. 总结`name_scope`与`variable_scope`的作用以及异同点

#name_scope的作用：
#1.解决功能近似的节点name可能命名重复的问题
#2.为其作用域中的节点的name添加一个或多个前缀，并使用这些前缀作为划分内部与外部op范围的标记，同时在TensorFlow可
    #视化时可以作为一个整体出现
#3.name_scope可以通过划分操作范围来组织图结构，并能服务于可视化
#4.name_scope可以给Op的name加前缀，但不包括tf.get_variable()创建的变量!!!!!!

#variable_scope的作用：
#1.主要用于管理变量作用域以及与变量相关的操作
#2.variable_scope也可以像name_scope一样用来给不同操作区域划分范围（添加name前缀）
#3.功能更为丰富，最重要的是可以与tf.get_variable()等配合使用完成对变量的重复使用

#相同点：
#1.都可以用来给不同操作区域划分范围（添加name前缀）
#2.在运行时均可返回一个上下文管理器

#不同点：
#1.name_scope不能给tf.get_variable()创建的变量加前缀，而variabel_scope可以与tf.get_variable()等配合使用
    #完成对变量的重复使用
#2.variable_scope包含了name_scope的全部功能，即在variable_scope下也可以给Op与Tensor加上name_scope
#3.变量作用域是可以嵌套使用的


#2. 构建逻辑回归模型（只有模型部分，不包括训练部分），使用`get_variable`与`variable_scope`
#将变量的创建与使用分开。提示：使用`tf.nn.sigmoid`实现`logistic`函数

import tensorflow as tf
#创建占位符
X=tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)
 
#创建变量
#tf.random_normal([1])返回一个符合正态分布的随机数
with tf.variable_scope('var',reuse=None) as var:
    tf.get_variable('w1',initializer=tf.random_normal([1],name='weight'))
    tf.get_variable('b1',initializer=tf.random_normal([1],name='bias'))
with tf.variable_scope('var',reuse=True) as var:
    w = tf.get_variable('w1')
    b = tf.get_variable('b1')

y_predict=tf.sigmoid(tf.add(tf.multiply(X,w),b))
num_samples=100
cost=tf.reduce_sum(tf.pow(y_predict-Y,2.0))/num_samples

In [10]:
#6.流程控制
#1. 设计一个函数，要求输入两个`shape、dtype`一样的张量，输出一个同样`shape、dtype`的张量，并且输出的张量中的元素的每一个值都是输入的
#两个张量中对应元素最大的。即模拟`tf.maximum`的功能，但不能直接使用此函数。
import tensorflow as tf
def my_maximum(x,y):
    shape = x.get_shape()
    maximum = tf.ones(shape=shape)
    #with tf.Session() as sess:
    #    rank = sess.run(tf.rank(x))
    with tf.Session() as sess:
        x = sess.run(x)
        y = sess.run(y)
        maximum = sess.run(maximum)
    for i in range(shape[0]):
        for j in range(shape[1]):
            if x[i][j]<y[i][j]:
                maximum[i][j]=y[i][j]
            else:
                maximum[i][j]=x[i][j]
    return maximum
x = tf.constant([[1,2],[3,4]])
y = tf.constant([[1,3],[2,4]])
maximum = my_maximum(x,y)
print(maximum)


[[1. 3.]
 [3. 4.]]


In [14]:
#2.了解`tf.minimum`、`tf.maximum`的用法

#tf.maximum：用法tf.maximum(a,b),返回的是a,b之间的最大值
#tf.miniimum：用法tf.miiinimum(a,b),返回的是a,b之间的最小值
import tensorflow as tf
a = [3,6,4] 
f1 = tf.maximum(a, 2) 
f2 = tf.minimum(a, 5) 
with tf.Session() as sess: 
    print(sess.run(f1))
    print(sess.run(f2)) 

[3 6 4]
[3 5 4]


In [16]:
#3. 了解`TensorArray`的基本用法，并尝试配合`tf.while_loop`配合使用。
import tensorflow as tf 
def cond(time, output_ta_l): 
    return tf.less(time, 3) 
def body(time, output_ta_l): 
    output_ta_l = output_ta_l.write(time, [6.5, 9.6]) 
    return time + 1, output_ta_l 
time = tf.constant(0) 
output_ta = tf.TensorArray(dtype=tf.float32, size=1, dynamic_size=True) 
result = tf.while_loop(cond, body, loop_vars=[time, output_ta]) 
last_time, last_out = result 
final_out = last_out.stack() 
with tf.Session(): 
    print(last_time.eval()) 
    print(final_out.eval())

3
[[6.5 9.6]
 [6.5 9.6]
 [6.5 9.6]]
